In [11]:
# IMPORTING LIBRARIES 
# Data handling
import os
import numpy as np 

# data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning tools
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout 
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix 

In [12]:
# DEFINING  DATASE PATH
base_dir = "dataset"
train_dir = os.path.join(base_dir, "training_set")
test_dir = os.path.join (base_dir, "testing_set")


In [13]:
# RESCALING IMAGES AND CRETING BATCHES AUTOMATICALLY 
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2 
)

train_generator  = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "binary",
    subset = "training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "binary",
    subset = "validation"
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    classes = ["testing_set"],
    class_mode = None,
    shuffle = False
)


Found 890 images belonging to 2 classes.
Found 222 images belonging to 2 classes.
Found 202 images belonging to 1 classes.


In [14]:
# BUILDING A CNN MODEL - VGG16 - a pretrained model on ImageNet 
vgg = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
for layer in vgg.layers:
    layer.trainable = False

model = Sequential ([
    vgg,
    Flatten(),
    Dense(256, activation = "relu"),
    Dropout (0.5),
    Dense (1, activation = "sigmoid")
])

model.compile (optimizer=Adam(learning_rate=0.0001),
              loss = "binary_crossentropy",
              metrics=["accuracy"])

model.summary 

2025-11-04 19:36:42.009433: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 101s 2us/step


<bound method Model.summary of <Sequential name=sequential, built=True>>

In [ ]:
# TRAIN THE MODEL 
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 10
)

Epoch 1/10


2025-11-04 19:39:49.965117: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 109051904 exceeds 10% of free system memory.
2025-11-04 19:39:50.707713: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 109051904 exceeds 10% of free system memory.
2025-11-04 19:39:53.869143: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 27262976 exceeds 10% of free system memory.
2025-11-04 19:39:53.969081: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 54525952 exceeds 10% of free system memory.
2025-11-04 19:39:55.479632: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 54525952 exceeds 10% of free system memory.


28/28 ━━━━━━━━━━━━━━━━━━━━ 1190s 43s/step - accuracy: 0.7191 - loss: 0.5790 - val_accuracy: 0.8018 - val_loss: 0.4552
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 993s 36s/step - accuracy: 0.8539 - loss: 0.3563 - val_accuracy: 0.8514 - val_loss: 0.3846
Epoch 3/10
 9/28 ━━━━━━━━━━━━━━━━━━━━ 12:19 39s/step - accuracy: 0.8286 - loss: 0.3888